<a href="https://colab.research.google.com/github/TONYPRIYAN/Advance_Java_Learning/blob/main/server_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# --- Final Model Server Script (Simplified & Corrected) ---

# 1. Install necessary libraries
!pip install gradio joblib scikit-learn pandas numpy xgboost imblearn

# 2. Import all libraries
import gradio as gr
import joblib
import pandas as pd
import numpy as np
import re
from google.colab import drive

# 3. Mount Google Drive
drive.mount('/content/drive')

# 4. Load your saved model and encoder
print("⏳ Loading all model components...")
try:
    pipeline_path = '/content/drive/MyDrive/Colab Notebooks/p1_p5_hybrid_classifier_pipeline.joblib'
    encoder_path = '/content/drive/MyDrive/Colab Notebooks/p1_p5_hybrid_label_encoder.joblib'
    loaded_pipeline = joblib.load(pipeline_path)
    loaded_encoder = joblib.load(encoder_path)
    print("✅ Models loaded successfully.")
except Exception as e:
    print(f"Error loading models: {e}")
    loaded_pipeline = None
    loaded_encoder = None

# 5. Define your final prediction function (same as before)
P1_KEYWORDS = ["outage", "fatal", "vulnerability", "data loss", "blocker", "corrupted", "revenue"]
P5_KEYWORDS = ["typo", "spelling", "dark mode", "enhancement", "suggestion", "nice to have", "copyright"]
CRITICAL_COMPONENTS = ["Core", "Database"]

def contains_keyword(text, keywords):
    return any(re.search(rf"\b{re.escape(kw)}\b", text) for kw in keywords)

def predict_priority_final(bug_title, bug_description, component):
    if not loaded_pipeline or not loaded_encoder:
        return {"prediction": "ERROR", "status": "Model not loaded"}
    # ... (rest of the function logic is the same as before)
    text_to_check = (bug_title + " " + bug_description).lower()
    if contains_keyword(text_to_check, P1_KEYWORDS):
        return {"prediction": "P1", "status": "Critical keyword detected"}
    if contains_keyword(text_to_check, P5_KEYWORDS):
        return {"prediction": "P5", "status": "Trivial keyword detected"}
    new_bug_df = pd.DataFrame({'text_input': [text_to_check], 'Component': [component]})
    prediction_probabilities = loaded_pipeline.predict_proba(new_bug_df)[0]
    original_probabilities = prediction_probabilities.copy()
    top_pred_index = np.argmax(prediction_probabilities)
    top_pred_label = loaded_encoder.inverse_transform([top_pred_index])[0]
    top_pred_confidence = prediction_probabilities[top_pred_index]
    status = "ML Prediction"
    final_prediction_label = top_pred_label
    if top_pred_label == 'P3' and top_pred_confidence < 0.80:
        status = f"P3 Conf low -> Overridden"
        prediction_probabilities[top_pred_index] = -1
        second_best_index = np.argmax(prediction_probabilities)
        final_prediction_label = loaded_encoder.inverse_transform([second_best_index])[0]
    p2_index = np.where(loaded_encoder.classes_ == 'P2')[0][0]
    if final_prediction_label == 'P2' and original_probabilities[p2_index] < 0.13:
        status += f" | P2 Conf low -> Escalated to P1"
        final_prediction_label = 'P1'
        p1_index = np.where(loaded_encoder.classes_ == 'P1')[0][0]
        p4_index = np.where(loaded_encoder.classes_ == 'P4')[0][0]
        if original_probabilities[p4_index] > original_probabilities[p1_index]:
            status += f" | P4 Conf > P1 Conf -> Corrected to P4"
            final_prediction_label = 'P4'
    if component in CRITICAL_COMPONENTS and final_prediction_label in ["P4", "P5"]:
        final_prediction_label = "P3"
        status = "Escalated due to critical component"
    return {"prediction": final_prediction_label, "status": status}

# 6. Create and launch the Gradio interface as a shareable API
print("\n🚀 Launching Gradio API...")
gr.Interface(
    fn=predict_priority_final,
    inputs=["text", "text", "text"],
    outputs="json"
).launch(share=True) # share=True is crucial for creating a public link

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⏳ Loading all model components...
✅ Models loaded successfully.

🚀 Launching Gradio API...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0cce99e772555d3b45.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
